In [2]:
import pandas as pd
train_path = "data/dataset.zip"

train_data = pd.read_csv(train_path, compression='zip', sep=',', index_col=0)

In [3]:
train_data.index.value_counts().to_dict()

{'39c16e': 2057, '3b9047': 2009, 'ebad26': 1996, '814d6b': 1103}

In [4]:
def oversample(df):
    classes = df.index.value_counts().to_dict()
    most = max(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df.index == key])
    classes_sample = []
    for i in range(1,len(classes_list)):
        classes_sample.append(classes_list[i].sample(most, replace=True))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[0]], axis=0)
    # final_df = final_df.reset_index(drop=True)
    return final_df

In [5]:
over_train_data = oversample(train_data)

In [6]:
over_train_data.index.value_counts().to_dict()

{'3b9047': 2057, 'ebad26': 2057, '814d6b': 2057, '39c16e': 2057}

# Tentativo di NER con spacy

## Utiliziamo NER sul nostro dataset

In [70]:
import spacy
import en_core_web_sm
from spacy import displacy

NER = en_core_web_sm.load()

In [33]:
over_train_data.head(2)

,text,content,wording,fixed_summary_text,prompt_question,prompt_title,prompt_text,prompt_text_preprocessed,length_ratio,normalized_text_length,...,corrected_misspelled_words,normalized_text_misspelled_counter,normalized_corrected_misspelled_counter,normalized_2grams-cooccurrence-count,normalized_2grams-correct-count,normalized_3grams-cooccurrence-count,normalized_3grams-correct-count,normalized_4grams-cooccurrence-count,normalized_4grams-correct-count,karp_tfidf_scores
prompt_id,,,,,,,,,,,,,,,,,,,,,
3b9047,"Some clasess ewere treated like real godesse, ...",-0.456956,-0.042516,"Some classes were treated like real goddess, w...","In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.090117,0.048602,...,set(),0.115385,0.0,0.060606,0.05227,0.051151,0.054092,0.049223,0.055866,0.403847
3b9047,"If you were a pharaoh, then that would mean yo...",-1.547163,-1.461245,"If you were a pharaoh, then that would mean yo...","In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...,egyptian society structured like pyramid top g...,0.047462,0.011497,...,set(),0.038462,0.0,0.012626,0.02751,0.007673,0.031900,0.005181,0.036313,0.053833


In [74]:
example = over_train_data['fixed_summary_text'][2]
print(example)

In summary the structure of the Ancient Egyptian system of government was shaped like a pyramid. In paragraph one, it states, "Egyptian society was structured like a pyramid." The different social classes were involved in this government by being ranked of the most worshipped to least worshipped.  It states, "At the top were the gods, such as Ra, Siris, and Isis." and "At the bottom of the social structure were slaves and farmers." These statements prove their loyalty to one another. 


In [75]:
first_index_text = NER(example)

In [78]:
print([(X, X.ent_iob_, X.ent_type_) for X in first_index_text])

[(In, 'O', ''), (summary, 'O', ''), (the, 'O', ''), (structure, 'O', ''), (of, 'O', ''), (the, 'O', ''), (Ancient, 'O', ''), (Egyptian, 'B', 'NORP'), (system, 'O', ''), (of, 'O', ''), (government, 'O', ''), (was, 'O', ''), (shaped, 'O', ''), (like, 'O', ''), (a, 'O', ''), (pyramid, 'O', ''), (., 'O', ''), (In, 'O', ''), (paragraph, 'O', ''), (one, 'O', ''), (,, 'O', ''), (it, 'O', ''), (states, 'O', ''), (,, 'O', ''), (", 'O', ''), (Egyptian, 'B', 'NORP'), (society, 'O', ''), (was, 'O', ''), (structured, 'O', ''), (like, 'O', ''), (a, 'O', ''), (pyramid, 'O', ''), (., 'O', ''), (", 'O', ''), (The, 'O', ''), (different, 'O', ''), (social, 'O', ''), (classes, 'O', ''), (were, 'O', ''), (involved, 'O', ''), (in, 'O', ''), (this, 'O', ''), (government, 'O', ''), (by, 'O', ''), (being, 'O', ''), (ranked, 'O', ''), (of, 'O', ''), (the, 'O', ''), (most, 'O', ''), (worshipped, 'O', ''), (to, 'O', ''), (least, 'O', ''), (worshipped, 'O', ''), (., 'O', ''), ( , 'O', ''), (It, 'O', ''), (states, 

In [79]:
displacy.render(first_index_text,style="ent",jupyter=True)

## NLTK for NER

In [80]:
# import modules and download packages
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# process the text and print Named entities
example_pre = over_train_data['correctedtext_preprocessed'][2]

# function
def get_named_entity(sent):
        try:
                sent = nltk.word_tokenize(sent)
                sent = nltk.pos_tag(sent)
                return sent
        except:
                pass
sent = get_named_entity(example_pre)

In [81]:
entities = {}
for s in sent:
    entities[s[1]].append(s[0]) if s[1] in entities else entities.update({s[1]: [s[0]]})

In [82]:
entities

{'IN': ['in',
  'of',
  'of',
  'like',
  'in',
  'like',
  'in',
  'by',
  'of',
  'at',
  'as',
  'at',
  'of'],
 'JJ': ['summary',
  'egyptian',
  'egyptian',
  'different',
  'social',
  'worshipped',
  'such',
  'ra',
  'social'],
 'DT': ['the',
  'the',
  'a',
  'a',
  'the',
  'this',
  'the',
  'the',
  'the',
  'the',
  'the',
  'these',
  'another'],
 'NN': ['structure',
  'ancient',
  'system',
  'government',
  'pyramid',
  'paragraph',
  'society',
  'pyramid',
  'government',
  'top',
  'siris',
  'isis',
  'bottom',
  'structure',
  'loyalty'],
 'VBD': ['was', 'was', 'were', 'were', 'were'],
 'VBN': ['shaped', 'structured', 'involved', 'ranked'],
 'CD': ['one', 'one'],
 'PRP': ['it', 'it'],
 'VBZ': ['states', 'states'],
 'NNS': ['classes', 'gods', 'slaves', 'farmers', 'statements'],
 'VBG': ['being'],
 'RBS': ['most'],
 'TO': ['to', 'to'],
 'JJS': ['least'],
 'VB': ['worshipped'],
 'CC': ['and', 'and', 'and'],
 'VBP': ['prove'],
 'PRP$': ['their']}